In [ ]:
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import json
import re
import html
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

## FILE PATHS

In [ ]:
partfilename = "part-r-00000"
metadata_filename = "data/meta_Video_Games.jsonl"
data_filename = "data/Video_Games_with_sentiment.jsonl"

## LOAD DATA

In [ ]:
data = []

with open(partfilename) as file:
    lines = file.readlines()
    for line in tqdm(lines):
        line = line.strip()
        keys, count = line.split("\t")
        year, month, sentiment, rating, asid = keys.split("-")

        # Add to list as a dictionary
        data.append({
            "year": year,
            "month": month,
            "sentiment": sentiment,
            "rating": rating,
            "asid": asid,
            "count": int(count)
        })

In [ ]:
# Create DataFrame from the list of dicts
df = pd.DataFrame(data)
df['year'] = df['year'].astype(int)
# Filter out years before 2010
# df = df[df['year'] >= 2010]
df.head()

In [ ]:
# Step 1: Load product metadata JSONL
with open(metadata_filename, "r") as f:
    metadata = [json.loads(line) for line in f]

# Step 2: Create mapping: parent_asin → title
title_map = {item["parent_asin"]: item["title"] for item in metadata}

# Step 3: Map titles to review DataFrame based on 'asid'
df["title"] = df["asid"].map(title_map)

# ✅ Optional: See the result
df[["asid", "title"]].head()

# Step 4: Remove rows with missing data
# df = df.dropna()

In [ ]:
top_products = (
    df.groupby("title")["count"].sum().sort_values(ascending=False).head(5)
)

print(top_products)

### WORD CLOUD

In [ ]:
# Load JSONL data
file =  open(data_filename, "r", encoding="utf-8")
lines = file.readlines()

In [ ]:
def get_review_text(lines):
    texts = []
    for line in tqdm(lines):
        try:
            review = json.loads(line)
            # Skip ratings
            if review["rating"] >= 3:
                continue
            raw_text = review["text"]
            clean_text = html.unescape(raw_text)
            clean_text = re.sub(r"<.*?>", " ", clean_text)
            clean_text = re.sub(r"[^a-zA-Z0-9\s']", " ", clean_text).lower()
            tokens = clean_text.split()
            tokens = [word for word in tokens if len(word) > 1 or word in ("i", "a")]
            texts.append(" ".join(tokens))
        except:
            continue  # skip malformed lines

    # Combine all review texts into one string
    all_text = " ".join(texts) 
    return all_text

In [ ]:
def get_title_text(lines):
    texts = []
    for line in tqdm(lines):
        try:
            review = json.loads(line)
            # Skip ratings
            if review["rating"] >= 3:
                continue
            raw_text = review["title"]
            clean_text = html.unescape(raw_text)
            clean_text = re.sub(r"<.*?>", " ", clean_text)
            clean_text = re.sub(r"[^a-zA-Z0-9\s']", " ", clean_text).lower()
            tokens = clean_text.split()
            tokens = [word for word in tokens if len(word) > 1 or word in ("i", "a")]
            texts.append(" ".join(tokens))
        except:
            continue  # skip malformed lines

    # Combine all review texts into one string
    all_text = " ".join(texts) 
    return all_text

In [ ]:
def create_wordcloud(all_text, title):
    # Generate the word cloud
    stopwords = set(STOPWORDS)

    # Generate the word cloud
    wordcloud = WordCloud(
        width=1000,
        height=600,
        background_color="white",
        stopwords=stopwords
    ).generate(all_text)

    # Show the word cloud
    plt.figure(figsize=(12, 6))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(title)
    plt.show()


In [ ]:
all_text = get_review_text(lines)
create_wordcloud(all_text, "Review Text")

In [ ]:
all_text = get_title_text(lines)

In [ ]:
create_wordcloud(all_text, "Title Text")

### RATING PIE CHART

In [ ]:
# 🧼 Filter the DataFrame
filtered_df = df

# Aggregate count by sentiment
# sentiment_counts = df.groupby("sentiment")["count"].sum()
sentiment_counts = filtered_df.groupby("rating")["count"].sum()

# Plot pie chart
plt.figure(figsize=(6, 6))
plt.pie(sentiment_counts, labels=sentiment_counts.index, autopct="%1.1f%%", startangle=140)
plt.title("Rating Distribution")
plt.axis("equal")
plt.show()

### SENTIMENT PIE CHART

In [ ]:
# 🔍 Choose the ASIN (product ID) to filter
# target_asid = "B00Y2CQRZY"  # <-- change this to any ASIN you want

# 🧼 Filter the DataFrame
# filtered_df = df[df["asid"] == target_asid]

# Aggregate count by sentiment
sentiment_counts = df.groupby("sentiment")["count"].sum()

# Plot pie chart
plt.figure(figsize=(6, 6))
wedges, texts, autotexts = plt.pie(
    sentiment_counts,
    labels=sentiment_counts.index,
    autopct="%1.1f%%",
    startangle=140
)

plt.title("Sentiment Distribution")
plt.axis("equal")

# ✅ Add legend
plt.legend(wedges, sentiment_counts.index, title="Sentiment", loc="lower right")
plt.show()

### BAR CHART

In [ ]:
import matplotlib.pyplot as plt

# Group by title and sum review counts
# product_counts = df.groupby("title")["count"].sum()
product_counts = df.groupby("title")["count"].sum()
# 🔍 Filter out products with fewer than 1000 reviews
filtered_counts = product_counts[product_counts >= 1000]

# Get Top and Bottom 10 products
top_10 = filtered_counts.nlargest(10)
bottom_10 = filtered_counts.nsmallest(10)

print("🔝 Top 10 Products by Review Count:")
print(top_10)
print("\n🔻 Bottom 10 Products by Review Count:")
print(bottom_10)

# Create subplot with 2 rows and 1 column
fig, axes = plt.subplots(2, 1, figsize=(10, 10))

# 🔝 Top 10 - Horizontal Bar Chart
axes[0].barh(top_10.index[::-1], top_10.values[::-1])  # Reverse for highest at top
axes[0].set_title("Top 10 Products by Review Count")
axes[0].set_xlabel("Review Count")
axes[0].set_ylabel("Product Title")

# 🔻 Bottom 10 - Horizontal Bar Chart
axes[1].barh(bottom_10.index, bottom_10.values, color="orange")
axes[1].set_title("Bottom 10 Products by Review Count")
axes[1].set_xlabel("Review Count")
axes[1].set_ylabel("Product Title")

# Layout and show
plt.tight_layout()
plt.show()

### YEARLY SALES

In [ ]:
df["year"].max()

In [ ]:
# Group by year and sum the review counts
yearly_sales = df.groupby("year")["count"].sum()

# Sort by year in case it's not already
yearly_sales = yearly_sales.sort_index()

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(yearly_sales.index, yearly_sales.values, marker='o', linestyle='-', linewidth=2)

# Labels and Title
plt.title("Change in Review Counts (as Sales Proxy) from 2010 to 2023")
plt.xlabel("Year")
plt.ylabel("Total Review Count")
plt.grid(True)

# Show plot
plt.tight_layout()
plt.show()


### RATING TRENDS

In [ ]:
# Group by year and sentiment, then sum the counts
sentiment_trends = df.groupby(["year", "rating"])["count"].sum().unstack()

# Sort by year to ensure proper plotting
sentiment_trends = sentiment_trends.sort_index()

# Plotting
plt.figure(figsize=(12, 6))
for sentiment in sentiment_trends.columns:
    plt.plot(sentiment_trends.index, sentiment_trends[sentiment], marker='o', label=sentiment)

# Labels and Title
plt.title("Rating Trends Over Time")
plt.xlabel("Year")
plt.ylabel("Review Count")
plt.legend(title="Rating")
plt.grid(True)
plt.tight_layout()
plt.show()

### XBOX/SONY/PLAYSTATION

In [ ]:
import json
import pandas as pd
from tqdm import tqdm
from datetime import datetime

data = []

with open(data_filename, "r", encoding="utf-8") as file:
    lines = file.readlines()

for line in tqdm(lines, desc="Processing reviews"):
    review = json.loads(line)

    # Extract and process fields
    rating = review.get("rating")
    title = review.get("title", "").lower()
    text = review.get("text", "").lower()
    sentiment = review.get("sentiment")
    asin = review.get("asin")
    timestamp = review.get("timestamp")

    # Convert timestamp to year and month
    if timestamp:
        dt = datetime.fromtimestamp(timestamp / 1000)
        year = dt.year
        month = dt.month
    else:
        year = None
        month = None

    # Combine title and text for keyword search
    combined_text = title + " " + text

    # Detect mentions
    mentions_xbox = any(kw in combined_text for kw in ["microsoft"])
    mentions_nintendo = any(kw in combined_text for kw in ["nintendo"])
    mentions_sony = any(kw in combined_text for kw in ["sony"])
    mentions_controller = any(kw in combined_text for kw in ["controller"])
    mentions_halo = any(kw in combined_text for kw in ["halo"])

    data.append({
        "year": year,
        "month": month,
        "rating": rating,
        "sentiment": sentiment,
        "asin": asin,
        "mentions_xbox": mentions_xbox,
        "mentions_nintendo": mentions_nintendo,
        "mentions_sony": mentions_sony,
        "mentions_controller": mentions_controller,
        "mentions_halo": mentions_halo,
    })

# Convert to DataFrame
extra_df = pd.DataFrame(data)


In [ ]:
extra_df.head()

In [ ]:
# 🧱 year と month から date 列を作成
extra_df["date"] = pd.to_datetime(extra_df[["year", "month"]].assign(day=1))

# 🗓️ 年月に変換
extra_df["year_month"] = extra_df["date"].dt.to_period("M")

# 📊 月別でメンション数を集計（True = 1, False = 0 としてカウントされる）
mentions_by_month = extra_df.groupby("year_month")[["mentions_xbox", "mentions_nintendo", "mentions_sony"]].sum()


# 📈 折れ線グラフを描画
plt.figure(figsize=(12, 6))
plt.plot(mentions_by_month.index.astype(str), mentions_by_month["mentions_xbox"], label="Xbox", color="green")
plt.plot(mentions_by_month.index.astype(str), mentions_by_month["mentions_nintendo"], label="Nintendo", color="red")
plt.plot(mentions_by_month.index.astype(str), mentions_by_month["mentions_sony"], label="Sony", color="blue")

plt.title("Brand Mentions by Month")
plt.xlabel("Month")
plt.ylabel("Number of Mentions")
plt.grid(True)
plt.legend()

# 🔍 X軸ラベルを間引いて見やすく
plt.xticks(
    ticks=range(0, len(mentions_by_month.index), 6),  # 6ヶ月ごと
    labels=mentions_by_month.index.astype(str)[::6],
    rotation=45
)

plt.tight_layout()
plt.show()


### Controller Reviews

In [ ]:
# 📅 year と month から date 列を作成
extra_df["date"] = pd.to_datetime(extra_df[["year", "month"]].assign(day=1))

# 🧼 年月 Period に変換
extra_df["year_month"] = extra_df["date"].dt.to_period("M")

# 🎮 コントローラーに言及したレビューのみ抽出
controller_df = extra_df[extra_df["mentions_controller"] == True]

# 📊 月 × sentiment ごとのレビュー件数をカウント
sentiment_counts = controller_df.groupby(["year_month", "sentiment"]).size().unstack(fill_value=0)

# 📈 折れ線グラフ描画
plt.figure(figsize=(12, 6))

for sentiment in sentiment_counts.columns:
    plt.plot(
        sentiment_counts.index.astype(str),
        sentiment_counts[sentiment],
        label=sentiment.capitalize()
    )

plt.title("Monthly Review Count by Sentiment (Controller Mentions Only)")
plt.xlabel("Year")
plt.ylabel("Number of Reviews")
plt.grid(True)
plt.legend(title="Sentiment")

# 🎯 毎年1月だけラベル表示
year_ticks = [i for i, p in enumerate(sentiment_counts.index) if p.month == 1]
year_labels = [str(p.year) for p in sentiment_counts.index if p.month == 1]
plt.xticks(ticks=year_ticks, labels=year_labels, rotation=0)

plt.tight_layout()
plt.show()


In [ ]:
# 📅 year と month から date 列を作成
extra_df["date"] = pd.to_datetime(extra_df[["year", "month"]].assign(day=1))

# 🧼 年月 Period に変換
extra_df["year_month"] = extra_df["date"].dt.to_period("M")

# 🎮 コントローラーに言及したレビューのみ抽出
controller_df = extra_df[extra_df["mentions_controller"] == True]

# 📊 月 × sentiment ごとのレビュー件数をカウント
ratings_count = controller_df.groupby(["year_month", "rating"]).size().unstack(fill_value=0)

# 📈 折れ線グラフ描画
plt.figure(figsize=(12, 6))

for rating in ratings_count.columns:
    plt.plot(
        ratings_count.index.astype(str),
        ratings_count[rating],
        label=rating
    )

plt.title("Monthly Review Count by Rating (Halo Mentions Only)")
plt.xlabel("Year")
plt.ylabel("Number of Reviews")
plt.grid(True)
plt.legend(title="Sentiment")

# 🎯 毎年1月だけラベル表示
year_ticks = [i for i, p in enumerate(ratings_count.index) if p.month == 1]
year_labels = [str(p.year) for p in ratings_count.index if p.month == 1]
plt.xticks(ticks=year_ticks, labels=year_labels, rotation=0)

plt.tight_layout()
plt.show()
